### Download Data from realtime website:  Declare site and # of files.  Run last cell FIRST

In [2]:
from gvradar import GVradar
import os, glob, sys, subprocess, requests
import pyart
from tqdm import trange
import imageio
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

# Set up local dirs
in_dir = './temp_files/'
out_dir = './cf_files/'
plt_dir = './plots/'
os.makedirs(in_dir, exist_ok=True)

# Declare site and # of files to grab
site = 'KILX'
f2p = 10


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119

Done.



### Download Data

In [3]:
# Download realtime data list
radar_info = requests.get("https://nomads.ncep.noaa.gov/pub/data/nccf/radar/nexrad_level2/" + site + "/dir.list")
with open("dir.list", "w") as f:
    f.write(radar_info.text)

# Read realtime list
file = open("dir.list", "r")
line_list = []
for line in file:
    line_list.append(line)

# Download files in range
for i in range(len(line_list) - f2p, len(line_list)):
    dfile = line_list[i].split()[1]
    # Download files
    radar_data = requests.get("https://nomads.ncep.noaa.gov/pub/data/nccf/radar/nexrad_level2/" + site + "/" + dfile)
    open(in_dir + dfile, "wb").write(radar_data.content)
    # Remove 2 from bz2 pyART issue
    bz_file = dfile[:-1]
    print('Downloading: ', bz_file)
    os.rename(in_dir + dfile, in_dir + bz_file)

Downloading:  KILX_20241004_144757.bz
Downloading:  KILX_20241004_145236.bz
Downloading:  KILX_20241004_145716.bz
Downloading:  KILX_20241004_150155.bz
Downloading:  KILX_20241004_150634.bz
Downloading:  KILX_20241004_151113.bz
Downloading:  KILX_20241004_151553.bz
Downloading:  KILX_20241004_152032.bz
Downloading:  KILX_20241004_152504.bz
Downloading:  KILX_20241004_152942.bz


### Run GVradar, run get_kwargs cell first.

In [4]:
run_DPQC = True
run_dp_products = False
remove_raw = True

wc = in_dir + '*.bz'
all_files = sorted(glob.glob(wc))
nf = len(all_files)
if(nf == 0):
    print("No files found in " + wc)
    sys.exit("Bye.")

for file in all_files:

    # Run DPQC (Quality Control)
    if run_DPQC:
        kwargs = get_kwargs()
        q = GVradar.QC(file, **kwargs)
        qc_radar = q.run_dpqc()

    # Run DP_products (rain rates, HID, DSD)
    if run_dp_products:
        kwargs_product = get_kwargs_product()
        d = GVradar.DP_products('QC_radar', qc_radar, **kwargs_product)
        d.run_DP_products()

    # Remove downloaded files
    if remove_raw:
        os.remove(file)


QC parameters:    

{'site': 'KILX', 'scan_type': 'PPI', 'month': '10', 'day': '04', 'year': '2024', 'hh': '14', 'mm': '47', 'ss': '57', 'ref_field_name': 'CZ', 'phi_field_name': 'PH', 'zdr_field_name': 'DR', 'radar_band': 'S', 'do_dbz': True, 'dbz_thresh': 5.0, 'do_rh': True, 'rh_thresh': 0.72, 'do_zdr': True, 'dr_min': -6.0, 'dr_max': 4.0, 'do_kdp': False, 'kdp_min': -2.0, 'kdp_max': 7.0, 'do_sq': False, 'sq_thresh': 0.45, 'do_sd': True, 'sd_thresh': 18.0, 'do_ph': False, 'ph_thresh': 80.0, 'dealias_velocity': False, 'merge_sp': False, 'do_ap': True, 'ap_dbz': 45, 'ap_zdr': 3, 'do_insect': False, 'do_despeckle': True, 'do_cos': False, 'coshmin': 0, 'coshmax': None, 'cosrmin': 0, 'cosrmax': 20, 'cosazmin': 0, 'cosazmax': 360, 'coselmin': 0, 'coselmax': 20.0, 'do_sector': False, 'sechmin': 0, 'sechmax': None, 'secrmin': 0, 'secrmax': 200, 'secazmin': 240, 'secazmax': 345, 'secelmin': 0, 'secelmax': 20.0, 'do_rh_sector': False, 'rhhmin': 0, 'rhhmax': None, 'rhrmin': 0, 'rhrmax': 50, 'r

### Make inline HTML movie of plots.

In [38]:
# Choose field to plot or Multi
field = 'CZ'

# Get list of plot files
png_files = []
filenames = sorted(glob.glob(os.path.join(plt_dir + '/' + field + '/' + site + '*.png')))

# Load images into list
images = []
for i in trange(len(filenames)):
    images.append(imageio.imread(filenames[i]))

# Use function to make the movie
anim = make_movie_mp4(images, site + '_latest.mp4', dpi=150)

# Convert to view video inline
a = HTML(anim.to_html5_video(embed_limit=None))
a

100%|██████████████████████████████████████████████████████████████| 14/14 [00:00<00:00, 103.30it/s]


<Figure size 704x594 with 0 Axes>

### **RUN FIRST** Input dictionaries for DPQC and DP_products and movie function.

In [37]:
# ***************************************************************************************

def get_kwargs():
    
    kwargs = {}

    # Set DBZ threshold, values less than thresh will be masked.
    kwargs.update({'do_dbz': True, 'dbz_thresh': 5.0})

    # Set RHOhv threshold, values less than thresh will be masked.
    kwargs.update({'do_rh': True, 'rh_thresh': 0.72})

    # Set ZDR threshold, values outside of range will be masked.   
    kwargs.update({'do_zdr': True, 'dr_min': -6.0, 'dr_max': 4.0})

    # Set KDP threshold, values outside of range will be masked.
    kwargs.update({'do_kdp': False, 'kdp_min': -2.0, 'kdp_max': 7.0})

    # Set SQ threshold, values less than thresh will be masked.
    kwargs.update({'do_sq': False, 'sq_thresh': 0.45})

    # Set SD threshold, values greater than thresh will be masked.
    kwargs.update({'do_sd': True, 'sd_thresh': 18.0})

    # Set PH threshold, values less than thresh will be masked.
    kwargs.update({'do_ph': False, 'ph_thresh': 80.0})

    # Choose if you would like to dealias_the velocity field.
    kwargs.update({'dealias_velocity': False, 'merge_sp': False}) 

    # Apply an AP filter, when DBZ is less then ap_dbz and ZDR is greater than ap_zdr, data will be masked.
    kwargs.update({'do_ap': True, 'ap_dbz': 45, 'ap_zdr': 3})

    # Apply CSU insect or despeckle filters.
    kwargs.update({'do_insect': False, 'do_despeckle': True})

    # Apply sector filters, Dual Pol thresholds can be applied to an user defined sector if needed.
    # Cone of silence filter, data within area will be masked.
    kwargs.update({'do_cos': False, 'coshmin': 0, 'coshmax': None, 'cosrmin': 0, 'cosrmax': 20,
                   'cosazmin': 0, 'cosazmax': 360, 'coselmin': 0, 'coselmax': 20.0})

    # Sector filter to masked all data, no thresholds needed.
    kwargs.update({'do_sector': False, 'sechmin': 0, 'sechmax': None, 'secrmin': 0, 'secrmax': 200,
                   'secazmin': 240, 'secazmax': 345, 'secelmin': 0, 'secelmax': 20.0})

    # Sector filter with RHOhv threshold, data in sector with values less than rh_sec will be masked.
    kwargs.update({'do_rh_sector': False, 'rhhmin': 0, 'rhhmax': None, 'rhrmin': 0, 'rhrmax': 50, 
                   'rhazmin': 0, 'rhazmax': 360, 'rhelmin': 0, 'rhelmax': 7.0, 'rh_sec': 0.92})

    # Sector filter with SD threshold, data in sector with values less than sd_sec will be masked.
    kwargs.update({'do_sd_sector': False, 'sdhmin': 0, 'sdhmax': None, 'sdrmin': 0, 'sdrmax': 75, 
                   'sdazmin': 0, 'sdazmax': 360, 'sdelmin': 0, 'sdelmax': 7.0, 'sd_sec': 8.0})

    # Sector filter with PH threshold, data in sector with values less than ph_sec will be masked.
    kwargs.update({'do_ph_sector': False, 'phhmin': 0, 'phhmax': None, 'phrmin': 0, 'phrmax': 200, 
                   'phazmin': 230, 'phazmax': 130, 'phelmin': 0, 'phelmax': 20.0, 'ph_sec': 80.0})

    # Apply calibration corrections if needed, both are subtracted.
    kwargs.update({'apply_cal': False, 'ref_cal': 0.2, 'zdr_cal': 0.0})

    ## There are two options for applying QC thresholds.

    # Apply QC thresholds based on height of the radar beam (km).
    # If True QC will be applied below qc_height.
    kwargs.update({'use_qc_height': True, 'qc_height': 4.4})

    # Apply QC based on the height of the freezing level.
    # If True, QC will only be applied below freeing level.
    kwargs.update({'use_sounding': False, 'sounding_type': 'ruc_archive',
                   'sounding_dir': './sounding/'})

    # Output CF file, fields to output, and output directory.
    kwargs.update({'output_cf': False,
                   'output_fields': ['DZ', 'CZ', 'VR', 'DR', 'KD', 'PH', 'RH', 'SD','SW'],
                   'cf_dir': './cf'})

    # Select plot limits, plot type (single or multiplot), and fields to plot.
    kwargs.update({'plot_images': True, 'max_range': 200, 'max_height': 200, 
                   'sweeps_to_plot': [0], 'plot_single': True, 'plot_multi': False,'add_logos': True, 
                   'fields_to_plot': ['CZ'], 'plot_raw_images': False, 'png': True,
                   'plot_dir': './plots/'})
    
    return kwargs

# ***************************************************************************************

def get_kwargs_product():
    
    kwargs_product = {}
    
    # Select if you want to output a cf file and what fields to write
    kwargs_product.update({'output_cf': False, 'cf_dir': './cf/',
                           'output_fields': ['DZ', 'CZ', 'VR', 'DR', 'KD',
                                             'PH', 'RH', 'SD', 'FS', 'FW',
                                             'RC', 'DM', 'NW', 'SQ']})
    kwargs_product.update({'output_grid': False, 'grid_dir': './grid/',
                           'output_fields': ['DZ', 'CZ', 'VR', 'DR', 'KD',
                                             'PH', 'RH', 'SD', 'FS',
                                             'RC', 'DM', 'NW']})

    # Select which products to produce.
    kwargs_product.update({'do_HID_summer': True,
                           'do_HID_winter': False,
                           'get_Bringi_kdp': False,
                           'do_mass': False,
                           'do_RC': True,
                           'do_RP': True,
                           'do_tokay_DSD': False,
                           'dsd_loc': 'all',
                           'do_150_mask': False,
                           'do_block_mask': False,
                           'get_cal_file': False, 'cal_dir': '/gvraid/trmmgv/caltxt_files/',
                           'apply_cal': False, 'ref_cal': 0.0, 'zdr_cal': 0.0})

    # Select plots ranges, type, and fields
    kwargs_product.update({'plot_images': True, 'plot_single': False, 'plot_multi': True,
                           'max_range': 200, 'max_height': 14, 'sweeps_to_plot': [0],
                           'fields_to_plot': ['CZ','RC','RP','FH'],
                           'plot_dir': './plots/', 'add_logos': False})

    # A Sounding is needed for DP products, sounding type can be; uwy, ruc, ruc_archive)
    kwargs_product.update({'use_sounding': True, 'sounding_type': 'get_ruc',
                           'sounding_dir': './soundings/'})

    return kwargs_product

# ***************************************************************************************

def make_movie_mp4(image_array, output_file, dpi):
    
    xpixels, ypixels = image_array[0].shape[0], image_array[0].shape[1]
    
    # make frames with each figure being the exact dimensions of our plots
    fig = plt.figure(figsize=(ypixels/dpi, xpixels/dpi), dpi=dpi)
    im = plt.figimage(image_array[0])

    def animate(i):
        im.set_array(image_array[i])
        return (im)
    
    mywriter = animation.FFMpegWriter(fps=3, bitrate=10000, codec='mpeg4', extra_args=['-pix_fmt', 'yuv420p'])
    anim = animation.FuncAnimation(fig, animate, frames=len(image_array), interval=200)
    anim.save(output_file,mywriter)
    
    return anim
    
# ***************************************************************************************